## Step 1: Check GPU Environment

Verifies GPU availability and CUDA version to ensure the environment is ready for Unsloth fine-tuning and subsequent llcuda deployment.

In [ ]:
import subprocess

print("="*70)
print("🔍 GPU ENVIRONMENT CHECK")
print("="*70)

# Check GPUs
!nvidia-smi --query-gpu=index,name,memory.total --format=csv

# CUDA version
print("\n📊 CUDA Version:")
!nvcc --version | grep release

print("\n✅ Environment ready for Unsloth + llcuda")

## Step 2: Install Unsloth and llcuda

Installs Unsloth for efficient fine-tuning, llcuda v2.2.0 for inference, and additional dependencies (datasets, trl) required for the training workflow.

In [ ]:
print("📦 Installing Unsloth and llcuda...")

# Install Unsloth (fast installation)
!pip install -q unsloth

# Install llcuda v2.2.0 (avoid --force-reinstall to prevent breaking RAPIDS/cupy)
!pip install -q git+https://github.com/llcuda/llcuda.git@v2.2.0

# Additional dependencies
!pip install -q datasets trl

# Verify installations
try:
    import llcuda
    print(f"\n✅ llcuda {llcuda.__version__} installed")
except Exception as e:
    print(f"❌ llcuda import failed: {e}")

try:
    from unsloth import FastLanguageModel
    print("✅ Unsloth installed")
except ImportError as e:
    print(f"⚠️ Unsloth import issue: {e}")

## Step 3: Load Base Model with Unsloth

We'll use Gemma-3 1B as it's fast to fine-tune on T4.

Loads Gemma-3 1B base model using Unsloth's FastLanguageModel with 4-bit quantization for memory-efficient fine-tuning on T4 GPU.

In [ ]:
%%time
from unsloth import FastLanguageModel
import torch

print("="*70)
print("📥 LOADING BASE MODEL WITH UNSLOTH")
print("="*70)

# Model configuration
model_name = "unsloth/gemma-3-1b-it"  # Small model for demo
max_seq_length = 2048

print(f"\n📥 Loading {model_name}...")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=torch.float16,
    load_in_4bit=True,  # Use 4-bit for training
)

print(f"\n✅ Model loaded!")
print(f"   Model: {model_name}")
print(f"   Max Sequence Length: {max_seq_length}")
print(f"   Precision: 4-bit")

## Step 4: Add LoRA Adapters

Adds LoRA (Low-Rank Adaptation) adapters to the model targeting attention and MLP layers, enabling parameter-efficient fine-tuning with only a small fraction of trainable parameters.

In [ ]:
print("="*70)
print("🔧 ADDING LORA ADAPTERS")
print("="*70)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,                # LoRA rank
    lora_alpha=32,       # LoRA alpha
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42,
)

# Count trainable parameters
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())

print(f"\n✅ LoRA adapters added!")
print(f"   Trainable params: {trainable:,} ({100*trainable/total:.2f}%)")
print(f"   Total params: {total:,}")

## Step 5: Prepare Training Dataset

Loads Alpaca dataset and formats it with instruction/input/response structure suitable for supervised fine-tuning, preparing 500 examples for demonstration.

In [ ]:
from datasets import load_dataset

print("="*70)
print("📊 PREPARING TRAINING DATASET")
print("="*70)

# Load a small dataset for demo (Alpaca format)
dataset = load_dataset("yahma/alpaca-cleaned", split="train[:500]")

print(f"\n📊 Dataset loaded: {len(dataset)} examples")
print(f"\n📋 Sample data:")
print(dataset[0])

# Format for training (Alpaca prompt format)
def format_alpaca(example):
    instruction = example.get("instruction", "")
    input_text = example.get("input", "")
    output = example.get("output", "")
    
    if input_text:
        prompt = f"""### Instruction:
{instruction}

### Input:
{input_text}

### Response:
{output}"""
    else:
        prompt = f"""### Instruction:
{instruction}

### Response:
{output}"""
    
    return {"text": prompt}

dataset = dataset.map(format_alpaca)
print(f"\n✅ Dataset formatted for training")

## Step 6: Train with SFTTrainer

Trains the model using SFTTrainer with optimized settings (8-bit AdamW, gradient accumulation, FP16) for 30 steps as a quick demonstration of the fine-tuning workflow.

In [ ]:
%%time
from trl import SFTTrainer, SFTConfig

print("="*70)
print("🏋️ TRAINING MODEL")
print("="*70)

# Training configuration (quick demo - increase for real training)
training_args = SFTConfig(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    max_steps=30,  # Quick demo - use more for real training
    learning_rate=2e-4,
    fp16=True,
    logging_steps=5,
    output_dir="./unsloth_output",
    optim="adamw_8bit",
    seed=42,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    processing_class=tokenizer,
    args=training_args,
)

print("\n🏋️ Starting training...")
print(f"   Batch size: {training_args.per_device_train_batch_size}")
print(f"   Max steps: {training_args.max_steps}")
print(f"   Learning rate: {training_args.learning_rate}")

trainer.train()

print("\n✅ Training complete!")

## Step 7: Export to GGUF Format

This is the key step - converting from Unsloth to llama.cpp compatible format.

Exports the fine-tuned model to GGUF format with Q4_K_M quantization using Unsloth's built-in export functionality, making it compatible with llama.cpp and llcuda.

In [ ]:
import os

print("="*70)
print("📦 EXPORTING TO GGUF FORMAT")
print("="*70)

# Output path
OUTPUT_DIR = "/kaggle/working/gguf_output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Unsloth's built-in GGUF export
print("\n📦 Exporting to GGUF with Q4_K_M quantization...")

model.save_pretrained_gguf(
    OUTPUT_DIR,
    tokenizer,
    quantization_method="q4_k_m",  # K-quant for best quality/size
)

# Find the exported file
gguf_files = [f for f in os.listdir(OUTPUT_DIR) if f.endswith('.gguf')]
print(f"\n✅ GGUF export complete!")
print(f"   Output directory: {OUTPUT_DIR}")
print(f"   Files: {gguf_files}")

if gguf_files:
    gguf_path = os.path.join(OUTPUT_DIR, gguf_files[0])
    size_mb = os.path.getsize(gguf_path) / (1024**2)
    print(f"   Size: {size_mb:.1f} MB")

Creates a compressed tar.gz archive of the GGUF model for easy download and distribution, with alternative download methods for Kaggle environment.

In [ ]:
# %% [markdown]
# ## Step 7.2: Create and Download GGUF Archive

# %%
import tarfile
import os
from google.colab import files

print("="*70)
print("📦 CREATING GGUF ARCHIVE")
print("="*70)

# Paths
GGUF_DIR = "/kaggle/working/gguf_output"
ARCHIVE_PATH = "/kaggle/working/fine-tuned-model.tar.gz"

# Check if GGUF directory exists and contains files
if not os.path.exists(GGUF_DIR):
    print(f"❌ GGUF directory not found: {GGUF_DIR}")
elif not os.listdir(GGUF_DIR):
    print(f"⚠️ GGUF directory is empty: {GGUF_DIR}")
else:
    print(f"📁 GGUF directory: {GGUF_DIR}")
    print("📋 Contents:")
    for item in os.listdir(GGUF_DIR):
        size = os.path.getsize(os.path.join(GGUF_DIR, item)) / (1024**2)
        print(f"   • {item} ({size:.1f} MB)")

# Create tar.gz archive
print(f"\n📦 Creating archive: {ARCHIVE_PATH}")
try:
    with tarfile.open(ARCHIVE_PATH, "w:gz") as tar:
        tar.add(GGUF_DIR, arcname=os.path.basename(GGUF_DIR))
    
    archive_size = os.path.getsize(ARCHIVE_PATH) / (1024**2)
    print(f"✅ Archive created: {ARCHIVE_PATH}")
    print(f"   Size: {archive_size:.1f} MB")
    
except Exception as e:
    print(f"❌ Failed to create archive: {e}")
    raise

# Download the archive
"""
print("\n⬇️  Downloading archive...")
try:
    # For Kaggle/Colab
    from google.colab import files
    files.download(ARCHIVE_PATH)
    print("✅ Download initiated!")
except ImportError:
    print("⚠️  Not in Colab environment. Archive saved locally:")
    print(f"   {ARCHIVE_PATH}")
except Exception as e:
    print(f"⚠️  Could not initiate download: {e}")
    print(f"📁 Archive available at: {ARCHIVE_PATH}")
"""


# Alternative: Provide download link in Kaggle
print("\n🔗 Alternative download methods:")
print(f"1. Direct path: {ARCHIVE_PATH}")
print("2. In Kaggle Notebooks, use the 'Data' tab on the right")
print("3. Copy to Google Drive:")
print(f"   !cp {ARCHIVE_PATH} /content/drive/MyDrive/")
print("4. Use kaggle API:")
print(f"   !kaggle datasets create -p {os.path.dirname(ARCHIVE_PATH)}")

## Step 8: Clear GPU Memory Before Inference

Clears GPU memory by deleting training objects and emptying CUDA cache to free up VRAM for deploying the fine-tuned model with llcuda.

In [ ]:
import gc
import torch

print("🧹 Clearing GPU memory...")

# Delete training objects
del model
del trainer
del tokenizer

# Clear CUDA cache
gc.collect()
torch.cuda.empty_cache()

print("\n📊 GPU Memory After Cleanup:")
!nvidia-smi --query-gpu=index,memory.used,memory.free --format=csv

print("\n✅ GPU memory cleared for inference")

## Step 9: Deploy with llcuda

Locates the exported GGUF file and deploys it using llcuda's ServerManager, starting llama-server with GPU acceleration for inference testing.

In [ ]:
# %% [markdown]
# ## Step 9: Deploy with llcuda

# %%
# Find GGUF file
import os
gguf_path = None
for root, dirs, files in os.walk("/kaggle/working"):
    for f in files:
        if f.endswith(".gguf") and "gemma" in f.lower():
            gguf_path = os.path.join(root, f)
            break
    if gguf_path:
        break

if gguf_path is None:
    raise FileNotFoundError("❌ Fine-tuned GGUF file not found!")

print(f"📥 Found fine-tuned model: {gguf_path}")

# %%
from llcuda.server import ServerManager

print("="*70)
print("🚀 DEPLOYING FINE-TUNED MODEL WITH LLCUDA")
print("="*70)

print(f"📥 Loading: {gguf_path}")

server = ServerManager()
print("\n🚀 Starting llama-server...")
try:
    server.start_server(
        model_path=gguf_path,
        host="127.0.0.1",
        port=8090,
        gpu_layers=32,      # GPU layers for acceleration
        ctx_size=2048,
        timeout=180,        # Increased timeout for startup
    )
except Exception as e:
    print(f"❌ Server failed to start: {e}")
    server.stop_server()
    raise

if server.check_server_health(timeout=180):
    print("\n✅ Fine-tuned model deployed with llcuda!")
    print(f"   API endpoint: http://127.0.0.1:8090")
    print(f"   Model: {os.path.basename(gguf_path)}")
    
    # Test the server
    print("\n🔍 Testing server health...")
    if server.check_server_health():
        print("   ✅ Server is responding")
    else:
        print("   ❌ Server not responding")
else:
    print("\n❌ Server failed to start")
    server.stop_server()

## Step 10: Test Your Fine-Tuned Model

Tests the fine-tuned model using both direct HTTP requests to llama-server and llcuda's client API, evaluating responses on Alpaca-formatted prompts and chat completions.

In [ ]:
## Step 10: Test Your Fine-Tuned Model

print("="*70)
print("🧪 TESTING FINE-TUNED MODEL")
print("="*70)

# Test with direct HTTP request to llama-server (most reliable)
import requests

test_prompts = [
    "### Instruction:\nExplain what machine learning is.\n\n### Response:",
    "### Instruction:\nWrite a short poem about coding.\n\n### Response:",
    "### Instruction:\nWhat are the benefits of GPU acceleration?\n\n### Response:",
]

for i, prompt in enumerate(test_prompts, 1):
    print(f"\n📋 Test {i}:")
    print(f"   Prompt: {prompt[:50]}...")
    
    try:
        response = requests.post(
            "http://127.0.0.1:8090/completion",
            json={
                "prompt": prompt,
                "max_tokens": 100,
                "temperature": 0.7,
                "stop": ["###", "\n\n"]
            },
            timeout=30
        )
        
        if response.status_code == 200:
            result = response.json()
            answer = result.get("content", "").strip()
            print(f"   ✅ Response: {answer[:200]}")
        else:
            print(f"   ❌ Error: HTTP {response.status_code}")
            
    except Exception as e:
        print(f"   ❌ Request failed: {e}")

print("\n" + "="*70)
print("📊 Testing with llcuda Client API")
print("="*70)

# Test with proper llcuda client API
try:
    from llcuda.api.client import LlamaCppClient
    
    client = LlamaCppClient(base_url="http://127.0.0.1:8090")
    
    # Test 1: Using chat completions (correct method)
    print("\n💬 Chat Completion Test:")
    chat_response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "What is machine learning?"}
        ],
        max_tokens=50
    )
    print(f"   ✅ Answer: {chat_response.choices[0].message.content[:200]}")
    
    # Test 2: Using completion endpoint with training format
    print("\n📝 Fine-tuned Format Test:")
    completion_response = requests.post(
        "http://127.0.0.1:8090/completion",
        json={
            "prompt": "What is machine learning?",
            "max_tokens": 50,
            "temperature": 0.7
        }
    )
    
    if completion_response.status_code == 200:
        result = completion_response.json()
        print(f"   ✅ Direct response: {result.get('content', '')[:200]}")
    
except Exception as e:
    print(f"   ⚠️  Error with client API: {e}")

print("\n🎯 Fine-tuned Model Successfully Tested!")
print("   ✅ Direct HTTP API: Working")
print("   🔗 Endpoint: http://127.0.0.1:8090/completion")
print("   📋 Format: Use Alpaca-style prompts for best results")

## Step 11: Compare with Chat API

Compares fine-tuned model performance between Alpaca instruction format and standard chat format, demonstrating compatibility with both /completion and /v1/chat/completions endpoints.

In [ ]:
## Step 11: Compare with Chat API

print("="*70)
print("💬 CHAT COMPLETION COMPARISON")
print("="*70)

# First, test with the original fine-tuned Alpaca format
print("\n🧪 Test 1: Fine-tuned Alpaca Format")
print("-" * 40)

alpaca_prompt = "### Instruction:\nWhat did you learn during fine-tuning?\n\n### Response:"

alpaca_response = requests.post(
    "http://127.0.0.1:8090/completion",
    json={
        "prompt": alpaca_prompt,
        "max_tokens": 150,
        "temperature": 0.7,
        "stop": ["###", "\n\n"]
    },
    timeout=30
)

if alpaca_response.status_code == 200:
    alpaca_result = alpaca_response.json()
    alpaca_answer = alpaca_result.get("content", "").strip()
    print(f"📝 Prompt: {alpaca_prompt[:60]}...")
    print(f"✅ Response: {alpaca_answer[:300]}")
    
    if "content" in alpaca_result:
        # Estimate tokens (rough approximation: 1 token ≈ 4 chars)
        prompt_tokens_approx = len(alpaca_prompt) // 4
        response_tokens_approx = len(alpaca_answer) // 4
        print(f"📊 Usage (approx):")
        print(f"   Prompt tokens: ~{prompt_tokens_approx}")
        print(f"   Completion tokens: ~{response_tokens_approx}")
else:
    print(f"❌ Error: HTTP {alpaca_response.status_code}")

# Second, test with standard chat format
print("\n\n🤖 Test 2: Standard Chat Format")
print("-" * 40)

try:
    from llcuda.api.client import LlamaCppClient
    client = LlamaCppClient(base_url="http://127.0.0.1:8090")
    
    chat_response = client.chat.completions.create(
        model="llama",  # Use generic model name
        messages=[
            {"role": "system", "content": "You are a helpful assistant fine-tuned on Alpaca dataset."},
            {"role": "user", "content": "What did you learn during fine-tuning?"}
        ],
        max_tokens=150,
        temperature=0.7
    )
    
    if hasattr(chat_response, 'choices') and len(chat_response.choices) > 0:
        chat_answer = chat_response.choices[0].message.content
        print(f"📝 Prompt: What did you learn during fine-tuning?")
        print(f"✅ Response: {chat_answer[:300]}")
        
        if hasattr(chat_response, 'usage'):
            print(f"📊 Usage (exact):")
            print(f"   Prompt tokens: {chat_response.usage.prompt_tokens}")
            print(f"   Completion tokens: {chat_response.usage.completion_tokens}")
        else:
            print(f"📊 Usage: Token information not available in response")
            
except Exception as e:
    print(f"⚠️  Chat API test failed: {e}")
    
    # Fallback to completion endpoint for chat
    print("\n🔄 Fallback: Using completion endpoint for chat-style")
    simple_prompt = "You are a helpful assistant. What did you learn during fine-tuning?"
    
    simple_response = requests.post(
        "http://127.0.0.1:8090/completion",
        json={
            "prompt": simple_prompt,
            "max_tokens": 150,
            "temperature": 0.7
        },
        timeout=30
    )
    
    if simple_response.status_code == 200:
        simple_result = simple_response.json()
        simple_answer = simple_result.get("content", "").strip()
        print(f"📝 Prompt: {simple_prompt[:60]}...")
        print(f"✅ Response: {simple_answer[:300]}")

print("\n" + "="*70)
print("📊 COMPARISON RESULTS")
print("="*70)
print("✅ Both Alpaca format and Chat format work with your fine-tuned model!")
print("📝 Tip: Use Alpaca format for best results (matches training data)")
print("🔗 API Endpoints:")
print("   - /completion : For Alpaca-style prompts")
print("   - /v1/chat/completions : For OpenAI-compatible chat")
print("\n🎯 Your llcuda server is fully functional!")

## Step 12: Save Model for Later Use

Creates a deployment-ready bundle by copying the GGUF model with descriptive naming, generating comprehensive metadata JSON, and packaging everything into a tar.gz archive with usage instructions.

In [ ]:
## Step 12: Save and Document Your Fine-Tuned Model

import shutil
import os
import json

print("="*70)
print("💾 SAVING AND DOCUMENTING FINE-TUNED MODEL")
print("="*70)

# Find the GGUF file (created by Unsloth export)
OUTPUT_DIR = "/kaggle/working/gguf_output"
gguf_files = [f for f in os.listdir(OUTPUT_DIR) if f.endswith('.gguf')]

if not gguf_files:
    # Search in working directory as backup
    for root, dirs, files in os.walk("/kaggle/working"):
        for f in files:
            if f.endswith(".gguf") and "gemma" in f.lower():
                gguf_files = [os.path.join(root, f)]
                break

if gguf_files:
    if isinstance(gguf_files[0], str) and os.path.isabs(gguf_files[0]):
        src = gguf_files[0]
    else:
        src = os.path.join(OUTPUT_DIR, gguf_files[0])
    
    # Create descriptive filename
    model_name = "my-finetuned-gemma-3-1b"
    dst = f"/kaggle/working/{model_name}-Q4_K_M.gguf"
    
    print(f"\n📂 Found GGUF file: {os.path.basename(src)}")
    print(f"   Source: {src}")
    print(f"   Size: {os.path.getsize(src) / (1024**2):.1f} MB")
    
    # Copy to working directory
    shutil.copy(src, dst)
    
    print(f"\n✅ Model saved to: {dst}")
    print(f"   Saved size: {os.path.getsize(dst) / (1024**2):.1f} MB")
    
    # Create model metadata file
    metadata = {
        "model_name": model_name,
        "base_model": "unsloth/gemma-3-1b-it",
        "quantization": "Q4_K_M",
        "training_data": "Alpaca-cleaned (500 samples)",
        "training_config": {
            "lora_rank": 16,
            "lora_alpha": 32,
            "max_seq_length": 2048,
            "batch_size": 2,
            "learning_rate": 2e-4,
            "trained_steps": 30
        },
        "export_format": "GGUF",
        "export_tool": "Unsloth + llama.cpp",
        "deployment_engine": "llcuda v2.2.0",
        "api_endpoints": {
            "completion": "http://127.0.0.1:8090/completion",
            "chat": "http://127.0.0.1:8090/v1/chat/completions"
        },
        "optimal_prompt_format": "### Instruction:\\n{instruction}\\n\\n### Response:",
        "file_size_mb": round(os.path.getsize(dst) / (1024**2), 2),
        "created": "2026-01-19",
        "environment": "Kaggle 2x Tesla T4"
    }
    
    metadata_path = f"/kaggle/working/{model_name}-metadata.json"
    with open(metadata_path, 'w') as f:
        json.dump(metadata, f, indent=2)
    
    print(f"\n📝 Metadata saved: {metadata_path}")
    
    # Create archive for easy download
    import tarfile
    archive_path = f"/kaggle/working/{model_name}-bundle.tar.gz"
    with tarfile.open(archive_path, "w:gz") as tar:
        tar.add(dst, arcname=os.path.basename(dst))
        tar.add(metadata_path, arcname=os.path.basename(metadata_path))
    
    print(f"\n📦 Archive created: {archive_path}")
    print(f"   Archive size: {os.path.getsize(archive_path) / (1024**2):.1f} MB")
    
    print("\n" + "="*70)
    print("🔧 USAGE INSTRUCTIONS")
    print("="*70)
    
    print("\n💡 Quick Start with llcuda:")
    print("```python")
    print("from llcuda.server import ServerManager")
    print("")
    print("# Start server")
    print("server = ServerManager()")
    print(f"server.start_server(model_path='{dst}', port=8090)")
    print("")
    print("# Test with Alpaca format (recommended)")
    print('prompt = "### Instruction:\\\\nYour question here\\\\n\\\\n### Response:"')
    print("import requests")
    print("response = requests.post('http://127.0.0.1:8090/completion',")
    print("  json={'prompt': prompt, 'max_tokens': 100, 'temperature': 0.7})")
    print("```")
    
    print("\n💡 Quick Start with llcuda Client API:")
    print("```python")
    print("from llcuda.api.client import LlamaCppClient")
    print("")
    print("client = LlamaCppClient(base_url='http://127.0.0.1:8090')")
    print("response = client.chat.completions.create(")
    print("  model='gemma-3-1b',")
    print("  messages=[{'role': 'user', 'content': 'Your question'}],")
    print("  max_tokens=100")
    print(")")
    print("```")
    
    print("\n📊 Model Performance:")
    print("   ✅ Supports both /completion and /v1/chat/completions endpoints")
    print("   ✅ Works with Alpaca-style prompts")
    print("   ✅ Token usage tracking available")
    print("   ✅ GPU accelerated with llcuda")
    
    print("\n🎯 Next Steps:")
    print("   1. Download the archive from Kaggle Data tab")
    print("   2. Deploy on any system with llcuda installed")
    print("   3. Use with Ollama: ollama create my-model -f ./Modelfile")
    print("   4. Share via HuggingFace Hub")
    
else:
    print("\n❌ No GGUF file found. Please check the export in Step 7.")
    print("   Expected in: /kaggle/working/ or /kaggle/working/gguf_output/")

## Step 13: Cleanup

Stops the llama-server and displays final GPU memory status to confirm all resources have been properly released after the fine-tuning and deployment workflow.

In [ ]:
print("🛑 Stopping server...")
server.stop_server()

print("\n✅ Server stopped")
print("\n📊 Final GPU Status:")
!nvidia-smi --query-gpu=index,memory.used,memory.free --format=csv

## 📚 Summary

### Complete Workflow:
1. ✅ Installed Unsloth + llcuda
2. ✅ Loaded base model with 4-bit quantization
3. ✅ Added LoRA adapters for efficient training
4. ✅ Fine-tuned on custom dataset
5. ✅ Exported to GGUF (Q4_K_M)
6. ✅ Deployed with llcuda llama-server
7. ✅ Ran inference on fine-tuned model

### Key llcuda + Unsloth Integration:

```python
from llcuda.unsloth import export_to_llcuda

# After Unsloth training
export_to_llcuda(
    model=model,
    tokenizer=tokenizer,
    output_path="my-model.gguf",
    quant_type="Q4_K_M"
)
```

---

**Next:** [06-split-gpu-graphistry](06-split-gpu-graphistry-llcuda-v2.2.0.ipynb)